# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
# !python -m pip install semantic-kernel==0.4.3.dev0

In [2]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the value of AZURE_OPENAI_DEPLOYMENT_GPT4
azure_openai_deployment_gpt4 = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT4")


In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment_name=azure_openai_deployment_gpt4, endpoint=endpoint, api_key=api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id))

## It all begins with an ask

In [4]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [5]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [6]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [7]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [8]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [9]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="shakespeare",
    skill_name="ShakespeareSkill",
    max_tokens=2000,
    temperature=0.8)

Let's update our ask using this new skill

In [10]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French but i want to write in English, but translate some romantic words to French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [11]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [12]:
results = await planner.execute_plan_async(new_plan, kernel)

In [13]:
print(results)

1. UN SOUPER D'AMOUR PRÉPARÉ DANS TA PROPRE CHEMINÉE, ÉCLAIRÉ PAR LA LUEUR DES BOUGIES ET AGRÉMENTÉ DE MÉLODIES DOUCES ET SUCRÉES.
2. UNE SOUDAINE ESCAPADE DE WEEK-END DANS UN LODGE PITTORESQUE DANS LES BOIS OU UN COTTAGE AU BORD DE LA MER.
3. PARTICIPER À UN COURS DE CUISINE OU DE POTERIE EN UNISSON, UNE EXPÉRIENCE PARTAGÉE.
4. ORGANISEZ UNE PROJECTION DE CONTES ROMANTIQUES DANS LE CONFORT DE VOTRE PROPRE DEMEURE.
5. ORGANISEZ UN ENVOI SOUDAIN DE FLEURS OU DE DOUCEURS À LEUR LIEU DE TRAVAIL.
6. RÉSERVEZ UNE SÉANCE DE SPA OU DE MASSAGE, UN RÉGAL POUR VOUS DEUX.
7. RENDEZ-VOUS DANS UNE CAVE OU UNE BRASSERIE LOCALE ET PARTICIPEZ À UNE DÉGUSTATION.
8. PLANIFIEZ UN VOYAGE À TRAVERS LE VERT PITTORESQUE OU UN PIQUE-NIQUE DANS UN PARC LOCAL.
9. ORGANISEZ UNE CÉLÉBRATION SOUDAINE AVEC DES PROCHES ET DES COMPAGNONS PROCHES DU CŒUR.
10. ORGANISEZ UN VOYAGE ROMANTIQUE EN MONTGOLFIÈRE OU UN VOYAGE EN BATEAU.


# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [14]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [15]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [16]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Generic function, unknown purpose : {'variables': {'input': ''}}
Translate the input into a language of your choice : {'variables': {'input': ''}}
Convert a string to uppercase. : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [17]:
result = await sequential_plan.invoke_async()

In [18]:
print(result)

1. AN EVE OF PASSION AT A FAVOURED TAVERN, SUCCEEDED BY A STROLL 'NEATH THE AZURE GLOW OF THE MOON.
2. A MEAL, PREPARED IN THE COMFORT OF OUR OWN HEARTH, ACCOMPANIED BY A TREASURED PLAY VIEWED IN TANDEM.
3. A WEEKEND RETREAT TO A SNUG LODGE WITHIN THE FOREST, OR A COASTAL INN.
4. AN UNEXPECTED RENDEZVOUS TO A CONCERT OR THEATRICAL PERFORMANCE.
5. A SESSION OF CULINARY OR PICTORIAL ARTISTRY, ENGAGED IN TOGETHER.
6. A FEAST IN THE PARK, PAIRED WITH A BELOVED TOME TO PERUSE AS ONE.
7. A TOUR OF TASTING THE FERMENTED GRAPE'S NECTAR IN A LOCAL VINEYARD.
8. A DAY'S JOURNEY TO A NEARBY CITY OR SPECTACLE OF INTEREST.
9. A DAY OF REJUVENATION IN THE BATHHOUSE, FILLED WITH MASSAGES AND LEISURE.
10. AN UNANTICIPATED GIFT OF A CHERISHED TOME, PLAY, OR JEWELLED TRINKET, FOLLOWED BY A ROMANTIC TWILIGHT WITHIN OUR OWN DWELLING.


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [19]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [20]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': SKFunction(),
 'trim': SKFunction(),
 'trim_end': SKFunction(),
 'trim_start': SKFunction(),
 'uppercase': SKFunction()}

In [21]:
ask = "What is the sum of 110 and 990?"

In [22]:
plan = await planner.create_plan_async(goal=ask)

In [23]:
for step in plan._steps:
    print(step.description, ":", step._state.__dict__)

In [24]:
result = await plan.invoke_async()

In [25]:
result.json()

C:\Users\mutaza\AppData\Local\Temp\ipykernel_32300\987439631.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  result.json()


'{"memory":{},"variables":{"variables":{"input":"1100","amount":"990"}},"skill_collection":{"data":{}}}'

In [26]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [27]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [28]:
# class WebSearchEngineSkill:
#     """
#     A search engine skill.
#     """
#     from semantic_kernel.orchestration.sk_context import SKContext
#     from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

#     def __init__(self, connector) -> None:
#         self._connector = connector

#     @sk_function(
#         description="Performs a web search for a given query", name="searchAsync"
#     )
#     @sk_function_context_parameter(
#         name="query",
#         description="The search query",
#     )
#     async def search_async(self, query: str, context: SKContext) -> str:
#         query = query or context.variables.get("query")[1]
#         result = await self._connector.search_async(query, num_results=5, offset=0)
#         return str(result)

In [29]:
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_skills import WebSearchEngineSkill

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': SKFunction()}

Let's also add a couple more skills

In [30]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': SKFunction(), 'Subtract': SKFunction()}

In [31]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)


Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [32]:
ask = """Find the area of Jordan in square kilometers. then multiply by 5, show your work"""

plan = planner.create_plan(goal=ask)

In [33]:
result = await plan.invoke_async()

Variable `$agent_scratch_pad` not found


In [34]:
print(result)

The area of Jordan is 91,880 square kilometers. When this is multiplied by 5, the result is 459,400 square kilometers.


Let's see the steps that the AI took to get to the answer.

In [35]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  To solve this problem, I need to first find the area of Jordan in square kilometers. I can use the WebSearch.searchAsync function to find this information. After obtaining the area, I will multiply it by 5 using the math.Add function. Let's start by finding the area of Jordan.
  [ACTION]
  {"action": "WebSearch.searchAsync", "action_variables": {"input": "Area of Jordan in square kilometers"}}
  [OBSERVATION]
  ['The territory of Jordan now covers about 91,880 square kilometres (35,480 sq mi).']
  [THOUGHT]
  The area of Jordan is 91,880 square kilometers. Now, I need to multiply this number by 5. I can use the math.Add function to do this.
  [ACTION]
  {"action": "math.Add", "action_variables": {"input": "91880", "Amount": "5"}}
  [OBSERVATION]
  91885
  [THOUGHT]
  None


In [36]:
search_func = kernel.skills.get_function("WebSearch", "searchAsync")

In [37]:
result = await search_func.invoke_async("who was the president of the United States in 2017??")
print(result)

["Upon the resignation of 37th president Richard Nixon, Gerald Ford became the 38th president even though he simply served out the remainder of Nixon's second term and was never elected to the presidency in his own right. Grover Cleveland was both the 22nd president and the 24th president because his two terms were not consecutive."]
